# RAG通用问答系统
---

## 1. 环境准备
- 安装依赖包
- 设置镜像源

In [ ]:
# 设置国内镜像源（加速下载）
import os

from langchain_core.output_parsers import StrOutputParser

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 安装依赖包（取消注释运行）
# !pip install langchain faiss-cpu huggingface-hub dashscope PyPDF2

## 2. 文档处理
- 加载PDF文件
- 检查加载结果

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

def load_documents(pdf_paths):
    """加载pdf文档并返回LangChain Document对象列表"""
    all_docs = []
    for path in pdf_paths:
        try:
            loader = PyPDFLoader(path)
            docs = loader.load()
            all_docs.extend(docs)
            print(f"成功加载：{path}（共{len(docs)}页）")
        except Exception as e:
            print(f"加载失败{path}：{str(e)}")
    return all_docs

# 示例使用
pdf_paths = ["pdf_China/中国产品质量法.pdf"]
processed_documents = load_documents(pdf_paths) # 原始文件
print(processed_documents)

## 3. 文本分割与向量化
- 中文文本分割
- 构建向量数据库

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(docs):
    """执行文本分割"""
    if not docs:
        raise ValueError("输入文档列表为空")

    # 中文优化分割器
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,     # 每个文本块500字符
        chunk_overlap=100,  # 块间重叠100字符
        separators=["\n\n", "\n", "。", "！", "？"] # 中文分隔符
    )

    # 执行分割
    split_docs = text_splitter.split_documents(docs)
    print(f"分割为{len(split_docs)}个文本块")

    # 查看前2个分割样例
    for i, doc in enumerate(split_docs[:2]):
        print(f"\n块{i+1}:\n{doc.page_content[:200]}...")

    return split_docs

# 示例使用
split_decs = split_documents(processed_documents)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def create_vector_store(split_docs):
    """创建并返回向量数据库"""
    if not split_docs:
        raise ValueError("无法用空文档创建向量库")

    embeddings = HuggingFaceEmbeddings(
        model_name="distiluse-base-multilingual-cased-v2"
    )

    vector_store = FAISS.from_documents(split_docs, embeddings)
    print(f"向量库已构建（包含{vector_store.index.ntotal}）个向量")

    return vector_store

# 示例使用
vector_store = create_vector_store(split_decs)

## 4. 通义千问配置
- 设置API密钥
- 初始化模型参数

In [ ]:
from langchain_community.llms import Tongyi
# import dashscope

# dashscope.api_key = "保密"
qwen_llm = Tongyi(model_name="qwen-turbo", temperature=0.3) # 使用通义千问的-turbo模型，调整模型稳定度为0.3


## 5. RAG核心链
- 设计提示模板
- 组合检索与生成模块

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def get_answer(question:str, vs, llm):
    """传统函数试实现（与RAG链区分）"""
    # 将向量库转换为检索器（具备检索功能）
    retriever = vs.as_retriever(search_kwargs={"k":3})

    # 执行检索
    retrieve_docs = retriever.invoke(question)

    # 构建提示词模板
    prompt = ChatPromptTemplate.from_template(
        """你是一个专家，请根据上下文回答问题：
        上下文：{context}
        问题：{question}
        专业回答："""
    )

    # 填充提示词模板
    formatted_prompt = prompt.format_messages(
        context=retrieve_docs,
        question=question
    )

    # 调用LLM
    llm_response = llm.invoke(formatted_prompt)

    # 解析输出
    return StrOutputParser().invoke(llm_response)

# 使用示例
answer = get_answer("中国产品质量法第一条是什么？", vector_store, qwen_llm)
print(answer)

## 6. 交互测试
- 实现问答循环
- 退出机制

In [ ]:
def interactive_qa_test(vector_store, llm):
    """
    传统函数式交互测试
    参数:
        vector_store: 已创建的向量数据库对象
        llm: 已初始化的语言模型
    """
    print("""\n🟢 法律问答系统(传统式)已启动
    ----------------------------------
    使用说明:
    1. 输入问题获取法律条文解答
    2. 输入以下命令可执行操作:
       /quit  - 退出系统
    ----------------------------------""")

    while True:
        try:
            # 获取用户输入
            user_input = input("\n❓ 请输入问题或命令: ").strip()

            if not user_input:
                print("⚠️ 输入不能为空，请重新输入")
                continue

            # 处理退出命令
            if user_input.lower() in ['/quit', '/exit', '/q']:
                print("🛑 系统已退出")
                break

            # 执行问答流程
            print("\n🔍 正在处理问题...")
            answer = get_answer(user_input, vector_store, llm)

            # 显示结果
            print(f"\n💡 答案: {answer}")
            print("-" * 50)  # 分隔线

        except KeyboardInterrupt:
            print("\n🛑 用户中断操作")
            break
        except Exception as e:
            print(f"\n❌ 发生错误: {str(e)}")

# ===== 使用示例 =====
# 在Notebook末尾单元格调用（需先初始化vector_store和qwen_llm）
if __name__ == "__main__":
    interactive_qa_test(vector_store, qwen_llm)